# Predicting Loan Defaults

Predicting loan defaults is an extremely common use case for machine learning in banking, one of DataRobot’s main target industries. As a loan officer, you are responsible for determining which loans are going to be the most profitable and worthy of lending money to. Based on a loan application from a potential client, you would like to predict whether the loan will be paid back in time.

## Data

You will be working with a loan dataset from LendingClub.com (source), a US peer-to-peer lending company. Download the dataset from the following URL:

 

https://s3.amazonaws.com/datarobot_public_datasets/DR_Demo_Lending_Club.csv

 

Your classification target is **is_bad**

# Task

* Partition your data into a holdout set and 5 stratified CV folds.
* Pick any two machine learning algorithms from the list below, and build a binary classification model with each of them:
  * Regularized Logistic Regression (scikit-learn)
  * Gradient Boosting Machine (scikit-learn, XGBoost or LightGBM)
  * Neural Network (Keras), with the architecture of your choice
* Both of your models must make use of numeric, categorical, text, and date features.
* Compute out-of-sample LogLoss and F1 scores on cross-validation and holdout.
* Which one of your two models would you recommend to deploy? Explain your decision.
* (Advanced, optional) Which 3 features are the most impactful for your model? Explain your methodology.

# Preprocessing & Exploratory Data Analysis

The Python code is configured to prune the dataset first so that redundant or uninformative features are not used in the training phase. By inspecting the data in a pandas dataframe, the following observations were made. 
* Column '_Id_' only contains sequential numbers and has no impact on prediction.
* Column '_pymnt__plan_' has only 2 'y' out of 10000 records. The rest of them are '_n_'.
* Column '_initial__list__status_' has only 17 values 'm' from 10000 records. The rest of them are 'f'.
* Column '_purpose_' is a textual representation and largly overlaps the information found in ‘purpose_cat’ so it was removed. The remaining are different only in few cases.
* Column 'collections_12_mnths_ex_med' was removed as it only has values 0 and NA

Therefore, these columns are removed by the code as a preprocessing step: 
* Id
* pymnt_plan
* initial_list_status
* purpose

# Categorical & Textual Data

The next step was preprocessing categorical and textual columns. All text was converted to lower case and punctuation was removed. Words like ‘company’, ‘corporation’ were removed from the *emp_title* value as they correspond to different ways of saying the same thing. In *emp_title* column variations of same employer were grouped together to form more representative categories for instance (‘us army’ and ‘us military’). Only categories with enough representation (more than 40 items per group) were kept for *emp_title*. For each record in *Notes* punctuation were removed and text was into words. Stop word list was generated to remove irrelevant wods. Words indicating various client parameters were left (stability of the job, education, missing payments) were kept as the reason for the loan is sometimes reflected in those words. Words with frequency higher than 400 were kept, each representing a separate feature. The feature is encoded by 1 if word is present and by 0 if word is not available. Column *earliest_cr_line* which is encoded as data were converted to numeric feature - number of days passed since that data.

For all the features on the training and prediction stage the following transformations were done:
* Numeric features were normalized by subtracting mean and dividing by standard deviation. In the encoded state missing values were replaced with 0 which represents mean value after encoding.
* Binary features (target column *is_bad* and all the columns representing words were kept as is.
* Categorical features were encoded with oneHot encoding (each category has its own column assigned to 1 for this category). Missing values has all 0s in those columns.


# Cross Validation

Two types of cross-validation were implemented: k-fold validation (value of folds can be selected in the **config.py**) and hold-out (percentage of the file used for training can be also set up in the **config.py**)

Algorithm first generates files which store corresponding training and prediction files. Next each fold algorithm is trained. Generated models are applied to remaining data for prediction. Results produced by each prediction step are combined together to form overall metrics for algorithm with this kind of parameters: accuracy, precision, recall, F1-score, logLoss. For accuracy, precision, recall and F1-score values are also generated for different thresholds of probability to see how these will affect prediction results.


# Importance Estimation

Three methods were used for importance estimation:

* Generation of the value based on variations in random tree leafs https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html

![Random Tree Leafs](importance_tree_1.png)

* After model is trained on the prediction stage each column (values of the feature) are randomly permuted. Accuracy of prediction on the permuted data is compared to the accuracy of prediction on the original data. Importance value is recorded as percentage of accuracy drop.

* After the model is trained on the prediction stage each column( values of the feature) are randomly permuted. LogLoss of prediction on the permuted data is compared to the LogLoss of prediction on the original data. Importance value is recorded as percentage of logloss change. It turned out that this method is not working well when working with the whole dataset. Features combined together replace each other and the change in the accuracy is not visible. LogLoss can change but still insignificantly. In order to select 3 most important features in this case sequential feature selection was applied. First combination of 1 feature was tested, logLoss was estimated and the value with the best logLoss was kept. Next pair of features including this previously selected one was tested. Finally, combinations of 3 most important features was generated. 
    * For logistic regression algorithm that is  ['revol_util', 'verification_status', 'annual_inc']
    * For gradient boosting that is: ['annual_inc', 'open_acc', 'total_acc']

Tests show that gradient boosting would be the best algorithm as it shows better values of recall.

# Configuration & Running the Code

It is easy to run the code from the command line: 

\>> python main.py

Please make sure your Python environment satisfies the following requirements: 

* numpy >= 1.14.5
* scipy >= 1.1.0
* pandas >= 0.23.1
* scikit-learn >= 0.19.1
* matplotlib >= 2.2.2

which can be found in requirements.txt.

The **main.py** file contains all the configuration for the run in the *main* section. Several things can be customized there. After running **main.py** with the default configuration, the output is the results of the prediction for the top three features. These are pasted at the end of this notebook. 

The classifiers are stored in the **loan_analysis/data** folder at the root level of the directory. The processed data is also saved there. 

# Function Architecture

I have tried to avoid using the sklearn libraries unless necessary. I did this so you can get a flavor of my coding style and to give you an idea of how I would approach a problem. Obviously, one can just use **sklearn.model_selection.StratifiedKFold()** instead of writing **crossValidation.py**. Here is a list of files with the functions they implement: 

* **axil.py** 
    * *clean_files()* utility to keep things tidy.
    * *check_extra_values()* checks if there are extra values in a column
    * *normalize_numeric()* normalizes a numeric column substracting its mean and dividing the result by standard deviation
* **config.py** the configuration file where the user is able to set the processing configuration parameters. 
* **crossValidation.py** 
    * *full_file_processing()* file processing utility for logistic regression or gradient boosting.
    * *crossValidation()* exactly as named, performs cross validation.
    * *cross_modelEvaluation()* evaluation function for logistic regression and gradient boosting.
* **data_reader.py** 
    * *read_Nstr_from_Csv()* reads specified number of rows from the csv file starting from selected position.
* **encoders_generation.py**
    * *generate_settings()* 
    * *get_categorical_encoders()* Returns categorical encoders according to data type prediction.
    * *get_missing_value_name()* Returns missing value name.
* **get_function_by_name.py**
    * *get_function_by_name()* Returns function (as a source) by its name and module name.
* **gradient_boosting_classifier.py**
    * *gradient_boosting_classifier_train()* Performs training using GradientBoostingClassfier()
    * *gradient_boosting_classifier_predict()* Performs prediction using GradientBoostingClassfier()
* **logistic_regression.py**
    * *logistic_regression_train()* Performs training using LogisticRegression()
    * *logistic_regression_predict()* Performs prediction using GradientBoostingClassfier()
* **main.py**
    * *check_stratification()*
    * *display_importances()*
    * __main__
* **preliminary_statistics.py**
    * *filter_employment()*
    * *filter_notes()*
    * *word_frequency()*
    * *clear_notes_using_dictionary()*
    * *clear_employment_using_dictionary()*
    * *purpose_handling_small_business()*
    * *date_conversion()*
    * *make_lower()*
    * *filter_text_and_categorical_values()*
* **refine_input_data.py**
    * *replace_name_property()*
    * *refineInputData()* 
* **save_load.py**
    * *save_obj()*
    * *save_objects()*
    * *load_obj()*
* **scores_computations.py**
    * *get_confusion_matrix()*
    * *accuracy_confusion_binary()*
    * *log_loss()*
* **split_for_validation.py**
    * *split_for_validation()*
    * *get_base_name()*
    * *generate_names()*
    * *correct_path()*
    * *split_data()*
    * *write_Partitions_Two()*
    * *write_Partitions_K()*
    * *write_To_Csv()*
* **tree_importance.py**
    * *get_tree_importance()*

Results of prediction using top three features are below: 

{'logistic_regression': {'thresholds': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'totalCount': 10000, 'algorithms': [{'algorithmName': 'logistic_regression', 'ACC': 0.8705, 'precision': 0, 'recall': 0.0, 'F1': 0, 'confusionLabel': [], 'importanceAccruacyNames': ['addr_state', 'policy_code', 'purpose_cat'], 'importanceAccuracyValues': [0.0, 0.0, 0.0], 'importanceLogLossNames': ['purpose_cat', 'policy_code', 'addr_state'], 'importanceLogLossValues': [39.680336207923396, 38.65302417441162, 38.596913788890504], 'accuracies': [0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8704, 0.8704, 0.8703, 0.8703, 0.8703, 0.8704, 0.8703, 0.8703, 0.8703, 0.8702, 0.87, 0.8701, 0.8701, 0.8701, 0.8693, 0.8684, 0.8673, 0.8664, 0.8654, 0.8637, 0.8625, 0.8586, 0.8562, 0.8533, 0.8506, 0.8478, 0.8409, 0.835, 0.8188, 0.7938, 0.7608, 0.6932, 0.6445, 0.5671, 0.4862, 0.3944, 0.3157, 0.2576, 0.21, 0.17, 0.1458, 0.1358, 0.1313, 0.1298, 0.1295, 0.1295, 0.1295], 'precisions': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007722007722007722, 0.0007722007722007722, 0.0007722007722007722, 0.0007722007722007722, 0.0007722007722007722, 0.0007722007722007722, 0.0015444015444015444, 0.0015444015444015444, 0.0023166023166023165, 0.005405405405405406, 0.010038610038610039, 0.011583011583011582, 0.019305019305019305, 0.030115830115830116, 0.03783783783783784, 0.044015444015444015, 0.05637065637065637, 0.06563706563706563, 0.07335907335907337, 0.07644787644787644, 0.08185328185328185, 0.09111969111969112, 0.10038610038610038, 0.12895752895752896, 0.18301158301158302, 0.2471042471042471, 0.35366795366795367, 0.4146718146718147, 0.5104247104247104, 0.6162162162162163, 0.7166023166023165, 0.8123552123552124, 0.8803088803088803, 0.9444015444015444, 0.9737451737451738, 0.9899613899613899, 0.9969111969111969, 1.0, 1.0, 1.0, 1.0, 1.0], 'recalls': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.25, 0.25, 0.25, 0.2, 0.14285714285714285, 0.25, 0.25, 0.3, 0.2692307692307692, 0.2765957446808511, 0.24193548387096775, 0.27472527472527475, 0.3023255813953488, 0.29518072289156627, 0.29381443298969073, 0.27547169811320754, 0.2715654952076677, 0.26243093922651933, 0.24937027707808565, 0.24145785876993167, 0.22180451127819548, 0.21138211382113822, 0.19623971797884843, 0.1909750201450443, 0.18422567645365573, 0.17032354034957234, 0.16106778644271147, 0.1517447199265381, 0.14671814671814673, 0.14024482393834065, 0.1374803972817564, 0.13556903317873706, 0.13512319080764557, 0.13236065917917497, 0.13066965650800122, 0.13002316446772083, 0.1297335203366059, 0.12953886165849754, 0.1295, 0.1295, 0.1295], 'F1s': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0015408320493066256, 0.0015396458814472672, 0.0015396458814472672, 0.0015396458814472672, 0.0015384615384615385, 0.0015360983102918587, 0.0030698388334612432, 0.0030698388334612432, 0.004597701149425287, 0.010598031794095384, 0.019374068554396422, 0.022107590272660283, 0.03607503607503607, 0.0547752808988764, 0.06707734428473648, 0.07656145063801209, 0.09358974358974359, 0.10572139303482588, 0.11466505733252867, 0.11702127659574466, 0.12226066897347172, 0.1291735084838533, 0.13612565445026178, 0.15563839701770738, 0.1869085173501577, 0.21108179419525067, 0.22991967871485947, 0.23201555411535973, 0.2339408954167404, 0.23700623700623702, 0.23458038422649138, 0.2351626243433553, 0.234954657873042, 0.23641987241445966, 0.2330437996673443, 0.23086619845128759, 0.23004276550249467, 0.22967101179391683, 0.22936592277718737, 0.22930500221336877, 0.22930500221336877, 0.22930500221336877], 'logLoss': 0.38227925380034483}]}, 'gradient_boosting_classifier': {'thresholds': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'totalCount': 10000, 'algorithms': [{'algorithmName': 'gradient_boosting_classifier', 'ACC': 0.8697, 'precision': 0.25, 'recall': 0.003088803088803089, 'F1': 0.006102212051868802, 'confusionLabel': [], 'importanceAccruacyNames': ['purpose_cat', 'addr_state', 'policy_code'], 'importanceAccuracyValues': [0.9083592043233298, 0.1264803955386915, 0.045992871104978725], 'importanceLogLossNames': ['purpose_cat', 'policy_code', 'addr_state'], 'importanceLogLossValues': [40.69472784385337, 39.152819692908245, 39.07873536008601], 'accuracies': [0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8706, 0.8706, 0.8706, 0.8705, 0.8705, 0.8705, 0.8704, 0.8704, 0.8704, 0.8704, 0.8703, 0.87, 0.87, 0.87, 0.87, 0.87, 0.8699, 0.8699, 0.8697, 0.8696, 0.8697, 0.8697, 0.8697, 0.8695, 0.8691, 0.8691, 0.8691, 0.8691, 0.8691, 0.8691, 0.8691, 0.8685, 0.8681, 0.8679, 0.8678, 0.8675, 0.8674, 0.8672, 0.8659, 0.8653, 0.8649, 0.8635, 0.8625, 0.8608, 0.8602, 0.8593, 0.8589, 0.8554, 0.8507, 0.8477, 0.8445, 0.8403, 0.8336, 0.8205, 0.8113, 0.7801, 0.7397, 0.6689, 0.5457, 0.4546, 0.3729, 0.3027, 0.2547, 0.2136, 0.1772, 0.1506, 0.1391, 0.1338, 0.1311, 0.13, 0.1295, 0.1295], 'precisions': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007722007722007722, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0023166023166023165, 0.0023166023166023165, 0.003088803088803089, 0.003088803088803089, 0.003088803088803089, 0.003088803088803089, 0.003088803088803089, 0.003088803088803089, 0.003088803088803089, 0.003088803088803089, 0.003088803088803089, 0.003088803088803089, 0.004633204633204633, 0.005405405405405406, 0.005405405405405406, 0.005405405405405406, 0.005405405405405406, 0.005405405405405406, 0.005405405405405406, 0.005405405405405406, 0.008494208494208495, 0.009266409266409266, 0.013127413127413128, 0.013899613899613899, 0.01698841698841699, 0.023166023166023165, 0.030115830115830116, 0.03397683397683398, 0.043243243243243246, 0.04942084942084942, 0.05714285714285714, 0.06563706563706563, 0.07567567567567568, 0.08416988416988418, 0.09343629343629344, 0.11505791505791506, 0.12818532818532818, 0.18301158301158302, 0.2525096525096525, 0.36602316602316604, 0.5181467181467182, 0.661003861003861, 0.7621621621621621, 0.8370656370656371, 0.8833976833976834, 0.9351351351351351, 0.9667953667953668, 0.9868725868725868, 0.9938223938223938, 0.9976833976833976, 0.9984555984555985, 1.0, 1.0, 1.0], 'recalls': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0.6666666666666666, 0.6666666666666666, 0.5, 0.5, 0.5, 0.4, 0.4, 0.4, 0.4, 0.3333333333333333, 0.2222222222222222, 0.2222222222222222, 0.2222222222222222, 0.2222222222222222, 0.2222222222222222, 0.2, 0.2, 0.21428571428571427, 0.2, 0.25, 0.25, 0.25, 0.2222222222222222, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.23076923076923078, 0.20588235294117646, 0.18421052631578946, 0.175, 0.17073170731707318, 0.1590909090909091, 0.15555555555555556, 0.14893617021276595, 0.16176470588235295, 0.15789473684210525, 0.18888888888888888, 0.16981132075471697, 0.1774193548387097, 0.1910828025477707, 0.2154696132596685, 0.22, 0.24561403508771928, 0.22939068100358423, 0.2138728323699422, 0.2135678391959799, 0.2149122807017544, 0.20961538461538462, 0.19803600654664485, 0.18671679197994986, 0.17965367965367965, 0.17198838896952104, 0.16666666666666666, 0.15991902834008098, 0.14618736383442266, 0.14580139669562256, 0.14201438848920864, 0.13815957175630894, 0.13544873312810798, 0.1346902458013569, 0.13266928049168167, 0.131009738595592, 0.13015776699029127, 0.12983619736709878, 0.12955911823647295, 0.1295647823911956, 0.1295, 0.1295], 'F1s': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0015432098765432098, 0.0030816640986132513, 0.0030816640986132513, 0.0030792917628945345, 0.0030792917628945345, 0.0030792917628945345, 0.003076923076923077, 0.003076923076923077, 0.003076923076923077, 0.003076923076923077, 0.0030745580322828594, 0.003067484662576687, 0.003067484662576687, 0.003067484662576687, 0.003067484662576687, 0.003067484662576687, 0.0030651340996168583, 0.0030651340996168583, 0.004583651642475171, 0.0045801526717557245, 0.006102212051868802, 0.006102212051868802, 0.006102212051868802, 0.006092916984006093, 0.006074411541381929, 0.006074411541381929, 0.006074411541381929, 0.006074411541381929, 0.006074411541381929, 0.006074411541381929, 0.009084027252081756, 0.01053423626787058, 0.010502625656414103, 0.010486891385767791, 0.010479041916167666, 0.010455563853622108, 0.010447761194029851, 0.010432190760059613, 0.016140865737344096, 0.0175054704595186, 0.02454873646209386, 0.025695931477516063, 0.031007751937984496, 0.04132231404958678, 0.052845528455284556, 0.05886287625418061, 0.07353906762967827, 0.08132147395171538, 0.09018890920170627, 0.10041346721795628, 0.11193603655054256, 0.12011019283746556, 0.12696747114375656, 0.14237935977066413, 0.14961694456962596, 0.17732884399551063, 0.20079828062634325, 0.22258746184550368, 0.22803738317757008, 0.23890594473904553, 0.23941782898726502, 0.23717317580133465, 0.23488348218868702, 0.235465681508847, 0.23332090942974285, 0.23131221719457015, 0.2301707949566306, 0.22977058509692339, 0.2293569844789357, 0.22940655447298494, 0.22930500221336877, 0.22930500221336877], 'logLoss': 0.38453881451537664}]}}
{'logistic_regression': {'thresholds': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'totalCount': 999, 'algorithms': [{'algorithmName': 'logistic_regression', 'ACC': 0.8688688688688688, 'precision': 0.0, 'recall': 0.0, 'F1': 0, 'confusionLabel': [], 'importanceAccruacyNames': ['addr_state', 'annual_inc', 'debt_to_income', 'delinq_2yrs', 'earliest_cr_line', 'emp_length', 'emp_title', 'home_ownership', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 'open_acc', 'policy_code', 'pub_rec', 'purpose_cat', 'revol_bal', 'revol_util', 'total_acc', 'verification_status', 'zip_code', 'word_help', 'word_stable', 'word_car', 'word_own', 'word_personal', 'word_purchase', 'word_good', 'word_period', 'word_high', 'word_over', 'word_fund', 'word_home', 'word_school', 'word_never', 'word_budget', 'word_company', 'word_payment', 'word_interest', 'word_lower', 'word_job', 'word_rate', 'word_debt', 'word_income', 'word_score', 'word_late', 'word_family', 'word_full', 'word_money', 'word_expenses', 'word_history', 'word_free', 'word_plan', 'word_balance', 'word_lend', 'word_business', 'word_club'], 'importanceAccuracyValues': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'importanceLogLossNames': ['addr_state', 'annual_inc', 'debt_to_income', 'delinq_2yrs', 'earliest_cr_line', 'emp_length', 'emp_title', 'home_ownership', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 'open_acc', 'policy_code', 'pub_rec', 'purpose_cat', 'revol_bal', 'revol_util', 'total_acc', 'verification_status', 'zip_code', 'word_help', 'word_stable', 'word_car', 'word_own', 'word_personal', 'word_purchase', 'word_good', 'word_period', 'word_high', 'word_over', 'word_fund', 'word_home', 'word_school', 'word_never', 'word_budget', 'word_company', 'word_payment', 'word_interest', 'word_lower', 'word_job', 'word_rate', 'word_debt', 'word_income', 'word_score', 'word_late', 'word_family', 'word_full', 'word_money', 'word_expenses', 'word_history', 'word_free', 'word_plan', 'word_balance', 'word_lend', 'word_business', 'word_club'], 'importanceLogLossValues': [38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275, 38.671814092857275], 'accuracies': [0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8698698698698699, 0.8688688688688688, 0.8688688688688688, 0.8688688688688688, 0.8688688688688688, 0.8688688688688688, 0.8688688688688688, 0.8698698698698699, 0.8708708708708709, 0.8708708708708709, 0.8698698698698699, 0.8688688688688688, 0.8688688688688688, 0.8678678678678678, 0.8658658658658659, 0.8658658658658659, 0.8648648648648649, 0.8638638638638638, 0.8628628628628628, 0.8638638638638638, 0.8618618618618619, 0.8598598598598599, 0.8578578578578578, 0.8538538538538538, 0.8458458458458459, 0.8368368368368369, 0.8318318318318318, 0.8268268268268268, 0.8238238238238238, 0.8188188188188188, 0.8118118118118118, 0.7997997997997998, 0.7807807807807807, 0.7607607607607607, 0.7407407407407407, 0.7157157157157157, 0.6946946946946947, 0.6556556556556556, 0.6136136136136137, 0.5785785785785785, 0.5425425425425425, 0.4914914914914915, 0.45045045045045046, 0.4174174174174174, 0.37237237237237236, 0.31431431431431434, 0.25225225225225223, 0.2172172172172172, 0.18518518518518517, 0.15015015015015015, 0.13413413413413414, 0.13013013013013014, 0.12912912912912913], 'precisions': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007751937984496124, 0.015503875968992248, 0.015503875968992248, 0.015503875968992248, 0.023255813953488372, 0.023255813953488372, 0.023255813953488372, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.03875968992248062, 0.05426356589147287, 0.05426356589147287, 0.06201550387596899, 0.06976744186046512, 0.07751937984496124, 0.08527131782945736, 0.08527131782945736, 0.11627906976744186, 0.11627906976744186, 0.13953488372093023, 0.15503875968992248, 0.17054263565891473, 0.20930232558139536, 0.24806201550387597, 0.27906976744186046, 0.31007751937984496, 0.3333333333333333, 0.3798449612403101, 0.40310077519379844, 0.4728682170542636, 0.5426356589147286, 0.5968992248062015, 0.6434108527131783, 0.6744186046511628, 0.751937984496124, 0.7674418604651163, 0.8372093023255814, 0.8837209302325582, 0.9147286821705426, 0.9457364341085271, 1.0, 1.0, 1.0, 1.0], 'recalls': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.5, 0.5, 0.4, 0.375, 0.375, 0.3333333333333333, 0.3076923076923077, 0.3076923076923077, 0.2857142857142857, 0.26666666666666666, 0.2777777777777778, 0.3333333333333333, 0.30434782608695654, 0.2962962962962963, 0.2903225806451613, 0.2702702702702703, 0.23404255319148937, 0.19642857142857142, 0.21739130434782608, 0.20270270270270271, 0.21686746987951808, 0.21739130434782608, 0.21359223300970873, 0.216, 0.2077922077922078, 0.1978021978021978, 0.19047619047619047, 0.17842323651452283, 0.17883211678832117, 0.16300940438871472, 0.16094986807387862, 0.16203703703703703, 0.15975103734439833, 0.15229357798165138, 0.14646464646464646, 0.14992272024729522, 0.14224137931034483, 0.13989637305699482, 0.1347517730496454, 0.1327334083239595, 0.13132400430570507, 0.13190184049079753, 0.12977867203219315, 0.12925851703406813, 0.12912912912912913], 'F1s': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.01515151515151515, 0.03007518796992481, 0.03007518796992481, 0.029850746268656712, 0.043795620437956206, 0.043795620437956206, 0.04347826086956522, 0.056338028169014086, 0.056338028169014086, 0.055944055944055944, 0.05555555555555555, 0.06802721088435375, 0.09333333333333334, 0.09210526315789475, 0.10256410256410256, 0.1125, 0.12048192771084336, 0.125, 0.11891891891891893, 0.15151515151515152, 0.14778325123152708, 0.169811320754717, 0.18099547511312214, 0.1896551724137931, 0.21259842519685038, 0.22614840989399296, 0.23151125401929257, 0.2359882005899705, 0.23243243243243245, 0.24317617866004965, 0.2321428571428571, 0.24015748031496065, 0.24955436720142601, 0.2520458265139116, 0.24629080118694363, 0.24066390041493776, 0.25, 0.24000000000000002, 0.23973362930077693, 0.23384615384615381, 0.2318271119842829, 0.23062381852551989, 0.23306233062330622, 0.22974176313446124, 0.22892635314995563, 0.22872340425531915], 'logLoss': 0.38671814092857276}]}, 'gradient_boosting_classifier': {'thresholds': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'totalCount': 999, 'algorithms': [{'algorithmName': 'gradient_boosting_classifier', 'ACC': 0.8778778778778779, 'precision': 0.8888888888888888, 'recall': 0.06201550387596899, 'F1': 0.11594202898550723, 'confusionLabel': [], 'importanceAccruacyNames': ['addr_state', 'annual_inc', 'debt_to_income', 'delinq_2yrs', 'earliest_cr_line', 'emp_length', 'emp_title', 'home_ownership', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 'open_acc', 'policy_code', 'pub_rec', 'purpose_cat', 'revol_bal', 'revol_util', 'total_acc', 'verification_status', 'zip_code', 'word_help', 'word_stable', 'word_car', 'word_own', 'word_personal', 'word_purchase', 'word_good', 'word_period', 'word_high', 'word_over', 'word_fund', 'word_home', 'word_school', 'word_never', 'word_budget', 'word_company', 'word_payment', 'word_interest', 'word_lower', 'word_job', 'word_rate', 'word_debt', 'word_income', 'word_score', 'word_late', 'word_family', 'word_full', 'word_money', 'word_expenses', 'word_history', 'word_free', 'word_plan', 'word_balance', 'word_lend', 'word_business', 'word_club'], 'importanceAccuracyValues': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'importanceLogLossNames': ['addr_state', 'annual_inc', 'debt_to_income', 'delinq_2yrs', 'earliest_cr_line', 'emp_length', 'emp_title', 'home_ownership', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 'open_acc', 'policy_code', 'pub_rec', 'purpose_cat', 'revol_bal', 'revol_util', 'total_acc', 'verification_status', 'zip_code', 'word_help', 'word_stable', 'word_car', 'word_own', 'word_personal', 'word_purchase', 'word_good', 'word_period', 'word_high', 'word_over', 'word_fund', 'word_home', 'word_school', 'word_never', 'word_budget', 'word_company', 'word_payment', 'word_interest', 'word_lower', 'word_job', 'word_rate', 'word_debt', 'word_income', 'word_score', 'word_late', 'word_family', 'word_full', 'word_money', 'word_expenses', 'word_history', 'word_free', 'word_plan', 'word_balance', 'word_lend', 'word_business', 'word_club'], 'importanceLogLossValues': [35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196, 35.581298840755196], 'accuracies': [0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8718718718718719, 0.8718718718718719, 0.8718718718718719, 0.8718718718718719, 0.8718718718718719, 0.8718718718718719, 0.8718718718718719, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8748748748748749, 0.8748748748748749, 0.8758758758758759, 0.8758758758758759, 0.8758758758758759, 0.8758758758758759, 0.8758758758758759, 0.8758758758758759, 0.8768768768768769, 0.8768768768768769, 0.8768768768768769, 0.8768768768768769, 0.8758758758758759, 0.8758758758758759, 0.8758758758758759, 0.8758758758758759, 0.8758758758758759, 0.8758758758758759, 0.8768768768768769, 0.8768768768768769, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8788788788788788, 0.8788788788788788, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8778778778778779, 0.8768768768768769, 0.8778778778778779, 0.8768768768768769, 0.8768768768768769, 0.8778778778778779, 0.8758758758758759, 0.8758758758758759, 0.8748748748748749, 0.8758758758758759, 0.8768768768768769, 0.8728728728728729, 0.8728728728728729, 0.8718718718718719, 0.8698698698698699, 0.8668668668668669, 0.8608608608608609, 0.8628628628628628, 0.8578578578578578, 0.8548548548548549, 0.8418418418418419, 0.8258258258258259, 0.8058058058058059, 0.7797797797797797, 0.7567567567567568, 0.7337337337337337, 0.6906906906906907, 0.6526526526526526, 0.6036036036036037, 0.5405405405405406, 0.47047047047047047, 0.4124124124124124, 0.35035035035035034, 0.2872872872872873, 0.21921921921921922, 0.17417417417417416, 0.14714714714714713, 0.13313313313313313, 0.12912912912912913, 0.12912912912912913, 0.12912912912912913], 'precisions': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007751937984496124, 0.007751937984496124, 0.007751937984496124, 0.007751937984496124, 0.007751937984496124, 0.007751937984496124, 0.007751937984496124, 0.023255813953488372, 0.023255813953488372, 0.023255813953488372, 0.023255813953488372, 0.031007751937984496, 0.031007751937984496, 0.03875968992248062, 0.03875968992248062, 0.03875968992248062, 0.03875968992248062, 0.03875968992248062, 0.03875968992248062, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.05426356589147287, 0.05426356589147287, 0.06201550387596899, 0.06201550387596899, 0.06201550387596899, 0.06201550387596899, 0.06201550387596899, 0.06201550387596899, 0.06201550387596899, 0.06976744186046512, 0.06976744186046512, 0.06976744186046512, 0.06976744186046512, 0.06976744186046512, 0.06976744186046512, 0.06976744186046512, 0.06976744186046512, 0.06976744186046512, 0.07751937984496124, 0.07751937984496124, 0.08527131782945736, 0.08527131782945736, 0.09302325581395349, 0.10077519379844961, 0.10852713178294573, 0.11627906976744186, 0.11627906976744186, 0.13178294573643412, 0.13953488372093023, 0.13953488372093023, 0.15503875968992248, 0.17054263565891473, 0.17054263565891473, 0.17829457364341086, 0.18604651162790697, 0.2248062015503876, 0.24031007751937986, 0.27906976744186046, 0.2868217054263566, 0.2868217054263566, 0.31007751937984496, 0.3488372093023256, 0.4186046511627907, 0.4806201550387597, 0.5271317829457365, 0.6046511627906976, 0.6589147286821705, 0.7209302325581395, 0.7441860465116279, 0.8062015503875969, 0.8372093023255814, 0.8837209302325582, 0.937984496124031, 0.9767441860465116, 0.9922480620155039, 1.0, 1.0, 1.0, 1.0], 'recalls': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.875, 0.875, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888, 0.9, 0.9, 0.8181818181818182, 0.8181818181818182, 0.8181818181818182, 0.8181818181818182, 0.8181818181818182, 0.8181818181818182, 0.8181818181818182, 0.7692307692307693, 0.7142857142857143, 0.7333333333333333, 0.6875, 0.6666666666666666, 0.6842105263157895, 0.6086956521739131, 0.6, 0.5769230769230769, 0.5862068965517241, 0.6, 0.5294117647058824, 0.5263157894736842, 0.5116279069767442, 0.4888888888888889, 0.46, 0.41379310344827586, 0.4393939393939394, 0.41333333333333333, 0.4090909090909091, 0.3592233009708738, 0.31092436974789917, 0.27586206896551724, 0.24861878453038674, 0.24324324324324326, 0.23754789272030652, 0.21518987341772153, 0.20855614973262032, 0.1945080091533181, 0.18023255813953487, 0.16216216216216217, 0.15615615615615616, 0.14673913043478262, 0.1405672009864365, 0.135498320268757, 0.13291139240506328, 0.13074565883554648, 0.12964824120603016, 0.12912912912912913, 0.12912912912912913, 0.12912912912912913], 'F1s': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.015384615384615384, 0.015384615384615384, 0.015384615384615384, 0.015384615384615384, 0.015384615384615384, 0.015384615384615384, 0.015384615384615384, 0.04545454545454545, 0.04545454545454545, 0.04545454545454545, 0.04545454545454545, 0.06015037593984962, 0.06015037593984962, 0.07462686567164178, 0.07462686567164178, 0.07462686567164178, 0.07462686567164178, 0.07462686567164178, 0.07462686567164178, 0.08888888888888888, 0.08888888888888888, 0.08888888888888888, 0.08888888888888888, 0.08823529411764706, 0.08823529411764706, 0.08823529411764706, 0.08823529411764706, 0.08823529411764706, 0.08823529411764706, 0.10218978102189781, 0.10218978102189781, 0.11594202898550723, 0.11594202898550723, 0.11594202898550723, 0.11594202898550723, 0.11594202898550723, 0.11594202898550723, 0.11594202898550723, 0.12949640287769784, 0.12949640287769784, 0.12857142857142856, 0.12857142857142856, 0.12857142857142856, 0.12857142857142856, 0.12857142857142856, 0.12857142857142856, 0.12857142857142856, 0.1408450704225352, 0.13986013986013984, 0.15277777777777776, 0.15172413793103448, 0.16326530612244897, 0.17567567567567569, 0.1842105263157895, 0.19480519480519481, 0.1935483870967742, 0.21518987341772153, 0.22641509433962265, 0.22085889570552145, 0.23952095808383236, 0.2558139534883721, 0.2528735632183908, 0.2569832402234637, 0.2566844919786096, 0.29743589743589743, 0.303921568627451, 0.3317972350230415, 0.31896551724137934, 0.2983870967741935, 0.291970802919708, 0.2903225806451613, 0.3076923076923077, 0.317948717948718, 0.3056179775280899, 0.3101391650099404, 0.30035335689045933, 0.2883720930232558, 0.26629680998613037, 0.26163522012578616, 0.24971098265895958, 0.2425531914893617, 0.2367906066536203, 0.23398328690807796, 0.23104693140794225, 0.22953736654804271, 0.22872340425531915, 0.22872340425531915, 0.22872340425531915], 'logLoss': 0.35581298840755193}]}}


{'logistic_regression': {'thresholds': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'totalCount': 999, 'algorithms': [{'algorithmName': 'logistic_regression', 'ACC': 0.8708708708708709, 'precision': 0, 'recall': 0.0, 'F1': 0,  'importanceAccruacyNames': ['revol_util', 'verification_status', 'annual_inc'], 'importanceAccuracyValues': [0.0, 0.0, 0.0], 'importanceLogLossNames': ['revol_util', 'verification_status', 'annual_inc'], 'importanceLogLossValues': [37.24814480982078, 37.24814480982078, 37.24814480982078], 'accuracies': [0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8678678678678678, 0.8668668668668669, 0.8648648648648649, 0.8578578578578578, 0.8368368368368369, 0.8138138138138138, 0.7867867867867868, 0.7467467467467468, 0.6926926926926927, 0.6226226226226226, 0.5595595595595596, 0.4924924924924925, 0.4244244244244244, 0.34434434434434436, 0.26926926926926925, 0.2032032032032032, 0.16016016016016016, 0.13713713713713713, 0.13413413413413414, 0.13113113113113112, 0.12912912912912913, 0.12912912912912913, 0.12912912912912913, 0.12912912912912913], 'precisions': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015503875968992248, 0.031007751937984496, 0.10077519379844961, 0.15503875968992248, 0.20930232558139536, 0.2713178294573643, 0.35658914728682173, 0.3953488372093023, 0.4728682170542636, 0.5658914728682171, 0.6046511627906976, 0.7131782945736435, 0.7829457364341085, 0.8527131782945736, 0.9147286821705426, 0.9612403100775194, 0.9689922480620154, 0.9922480620155039, 0.9922480620155039, 0.9922480620155039, 1.0, 1.0, 1.0, 1.0], 'recalls': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2857142857142857, 0.3333333333333333, 0.40625, 0.37735849056603776, 0.3068181818181818, 0.2755905511811024, 0.26136363636363635, 0.22566371681415928, 0.20333333333333334, 0.18527918781725888, 0.1670235546038544, 0.16370106761565836, 0.1558641975308642, 0.14745308310991956, 0.14097968936678615, 0.1355191256830601, 0.13020833333333334, 0.12942366026289182, 0.12903225806451613, 0.12864321608040202, 0.12912912912912913, 0.12912912912912913, 0.12912912912912913, 0.12912912912912913], 'F1s': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.02941176470588235, 0.05673758865248227, 0.16149068322981366, 0.2197802197802198, 0.24884792626728114, 0.2734375, 0.30163934426229505, 0.28732394366197184, 0.2843822843822844, 0.2791586998087954, 0.26174496644295303, 0.2662807525325615, 0.25997425997425994, 0.2514285714285714, 0.2443064182194617, 0.23754789272030652, 0.22956841138659326, 0.22898032200357785, 0.22836752899197146, 0.22775800711743774, 0.22872340425531915, 0.22872340425531915, 0.22872340425531915, 0.22872340425531915], 'logLoss': 0.37248144809820777}]}, 'gradient_boosting_classifier': {'thresholds': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'totalCount': 999, 'algorithms': [{'algorithmName': 'gradient_boosting_classifier', 'ACC': 0.8738738738738738, 'precision': 0.8, 'recall': 0.031007751937984496, 'F1': 0.059701492537313425, 'importanceAccruacyNames': ['annual_inc', 'total_acc', 'open_acc'], 'importanceAccuracyValues': [8.132875143184421, 5.612829324169531, 5.154639175257731], 'importanceLogLossNames': ['annual_inc', 'open_acc', 'total_acc'], 'importanceLogLossValues': [54.11493136487354, 48.871411616970825, 48.826434423828275], 'accuracies': [0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8708708708708709, 0.8728728728728729, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8738738738738738, 0.8748748748748749, 0.8758758758758759, 0.8758758758758759, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8748748748748749, 0.8738738738738738, 0.8718718718718719, 0.8718718718718719, 0.8718718718718719, 0.8698698698698699, 0.8698698698698699, 0.8688688688688688, 0.8658658658658659, 0.8618618618618619, 0.8518518518518519, 0.8498498498498499, 0.8408408408408409, 0.8328328328328328, 0.8208208208208209, 0.8048048048048048, 0.7797797797797797, 0.7347347347347347, 0.6546546546546547, 0.6206206206206206, 0.5365365365365365, 0.45345345345345345, 0.33433433433433435, 0.24724724724724725, 0.18518518518518517, 0.15115115115115116, 0.13713713713713713, 0.13113113113113112, 0.13013013013013014, 0.12912912912912913, 0.12912912912912913, 0.12912912912912913, 0.12912912912912913], 'precisions': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015503875968992248, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.031007751937984496, 0.03875968992248062, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.046511627906976744, 0.05426356589147287, 0.06201550387596899, 0.07751937984496124, 0.07751937984496124, 0.07751937984496124, 0.07751937984496124, 0.08527131782945736, 0.08527131782945736, 0.09302325581395349, 0.10077519379844961, 0.15503875968992248, 0.20155038759689922, 0.3798449612403101, 0.4883720930232558, 0.5891472868217055, 0.7364341085271318, 0.8604651162790697, 0.9069767441860465, 0.9922480620155039, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'recalls': [0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8333333333333334, 0.8571428571428571, 0.8571428571428571, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.6666666666666666, 0.5454545454545454, 0.5454545454545454, 0.5454545454545454, 0.46153846153846156, 0.4666666666666667, 0.4444444444444444, 0.4, 0.3448275862068966, 0.2564102564102564, 0.24390243902439024, 0.21153846153846154, 0.18333333333333332, 0.16216216216216217, 0.14130434782608695, 0.15267175572519084, 0.13829787234042554, 0.15605095541401273, 0.1675531914893617, 0.15637860082304528, 0.15650741350906094, 0.14643799472295516, 0.13652275379229872, 0.13602550478214664, 0.13203684749232344, 0.1301715438950555, 0.12938816449348045, 0.12925851703406813, 0.12912912912912913, 0.12912912912912913, 0.12912912912912913, 0.12912912912912913], 'F1s': [0, 0, 0, 0, 0, 0, 0, 0.030534351145038167, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.06015037593984962, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.059701492537313425, 0.07407407407407408, 0.08823529411764706, 0.08823529411764706, 0.08759124087591241, 0.08759124087591241, 0.08759124087591241, 0.08759124087591241, 0.08759124087591241, 0.08759124087591241, 0.08759124087591241, 0.08759124087591241, 0.08759124087591241, 0.08759124087591241, 0.08759124087591241, 0.08759124087591241, 0.08695652173913045, 0.08571428571428572, 0.08571428571428572, 0.08571428571428572, 0.08450704225352114, 0.09722222222222222, 0.10884353741496598, 0.12987012987012986, 0.12658227848101267, 0.11904761904761903, 0.1176470588235294, 0.12154696132596683, 0.11640211640211638, 0.11822660098522167, 0.11764705882352941, 0.15384615384615385, 0.1640378548895899, 0.22121896162528218, 0.24950495049504948, 0.24715447154471545, 0.25815217391304346, 0.2502818489289741, 0.23732251521298176, 0.2392523364485981, 0.2332730560578662, 0.23035714285714287, 0.22912966252220252, 0.22892635314995563, 0.22872340425531915, 0.22872340425531915, 0.22872340425531915, 0.22872340425531915], 'logLoss': 0.37435263296842963}]}}

{'logistic_regression': {'thresholds': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'totalCount': 10000, 'algorithms': [{'algorithmName': 'logistic_regression', 'ACC': 0.8705, 'precision': 0, 'recall': 0.0, 'F1': 0,'importanceAccruacyNames': ['revol_util', 'verification_status', 'annual_inc'], 'importanceAccuracyValues': [0.0, 0.0, 0.0], 'importanceLogLossNames': ['revol_util', 'verification_status', 'annual_inc'], 'importanceLogLossValues': [37.88511232024727, 37.88511232024727, 37.88511232024727], 'accuracies': [0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8705, 0.8704, 0.8701, 0.8698, 0.8694, 0.8683, 0.8677, 0.8661, 0.8629, 0.8589, 0.8509, 0.8411, 0.8265, 0.7986, 0.7687, 0.7354, 0.6892, 0.6316, 0.5676, 0.4945, 0.4136, 0.3411, 0.2729, 0.2176, 0.1783, 0.1537, 0.1421, 0.136, 0.1331, 0.1313, 0.1301, 0.1296], 'precisions': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007722007722007722, 0.003088803088803089, 0.0069498069498069494, 0.013127413127413128, 0.022393822393822392, 0.03474903474903475, 0.04633204633204633, 0.06023166023166023, 0.09575289575289575, 0.14362934362934363, 0.1891891891891892, 0.24864864864864866, 0.31891891891891894, 0.4038610038610039, 0.4980694980694981, 0.5907335907335908, 0.6880308880308881, 0.7675675675675676, 0.833976833976834, 0.8988416988416988, 0.9444015444015444, 0.9683397683397683, 0.9861003861003861, 0.993050193050193, 0.9945945945945946, 0.9953667953667954, 0.9969111969111969, 0.9976833976833976, 1.0], 'recalls': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.1111111111111111, 0.21052631578947367, 0.225, 0.27419354838709675, 0.28431372549019607, 0.2710843373493976, 0.2542372881355932, 0.2215909090909091, 0.22878228782287824, 0.229064039408867, 0.20264681555004135, 0.19374247894103488, 0.18971061093247588, 0.18293109478838754, 0.17531938026637672, 0.16779995613073043, 0.16077228437387225, 0.15159371663870672, 0.14488865038905285, 0.14017341040462428, 0.13626740947075208, 0.1329798515376458, 0.13135157375025716, 0.1304788961038961, 0.12982562241709505, 0.1295217041800643, 0.12943653499097654, 0.12935522627152582, 0.12951295129512952], 'F1s': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0015337423312883436, 0.006088280060882801, 0.013483146067415729, 0.025055268975681652, 0.04151753758052971, 0.06160164271047228, 0.0783801436969301, 0.09471766848816028, 0.13500272182906914, 0.17655434266729947, 0.19568690095846647, 0.21778829895164015, 0.23790322580645162, 0.2518054886856042, 0.25934861278648974, 0.2613597540143492, 0.2606406318560772, 0.2531839021905247, 0.2468853583266659, 0.24252526304823419, 0.23816942551119766, 0.23384615384615384, 0.23182354542979036, 0.23065195946551875, 0.2296718972895863, 0.22921668000355647, 0.2291241458869465, 0.22901710537977485, 0.22932530547193203], 'logLoss': 0.3788511232024728}]}, 'gradient_boosting_classifier': {'thresholds': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'totalCount': 10000, 'algorithms': [{'algorithmName': 'gradient_boosting_classifier', 'ACC': 0.8733, 'precision': 0.6590909090909091, 'recall': 0.044787644787644784, 'F1': 0.08387563268257411, 'importanceAccruacyNames': ['annual_inc', 'open_acc', 'total_acc'], 'importanceAccuracyValues': [7.133860070995077, 6.297950303446696, 6.22924539104546], 'importanceLogLossNames': ['annual_inc', 'open_acc', 'total_acc'], 'importanceLogLossValues': [49.905440558773165, 47.83117288407886, 47.62432536635895], 'accuracies': [0.8705, 0.8705, 0.8706, 0.8706, 0.8707, 0.8707, 0.8707, 0.8708, 0.8708, 0.8711, 0.8712, 0.8716, 0.8721, 0.8722, 0.8723, 0.8727, 0.8734, 0.8735, 0.8735, 0.8736, 0.8738, 0.8739, 0.8744, 0.8745, 0.8748, 0.875, 0.8751, 0.8751, 0.8751, 0.8751, 0.8751, 0.8752, 0.8751, 0.8753, 0.8753, 0.8752, 0.8751, 0.8751, 0.8751, 0.875, 0.8749, 0.8748, 0.8747, 0.8748, 0.8747, 0.8748, 0.8745, 0.8743, 0.874, 0.8739, 0.8733, 0.8732, 0.8727, 0.8727, 0.8726, 0.8726, 0.8725, 0.8725, 0.8726, 0.8726, 0.8717, 0.8706, 0.8699, 0.8695, 0.8689, 0.8687, 0.8685, 0.8683, 0.8678, 0.8673, 0.8665, 0.866, 0.8649, 0.8631, 0.8606, 0.8587, 0.857, 0.8539, 0.8514, 0.8474, 0.8432, 0.8353, 0.8245, 0.8104, 0.791, 0.7513, 0.6894, 0.612, 0.5256, 0.4533, 0.3612, 0.2848, 0.2218, 0.1822, 0.1543, 0.1401, 0.1344, 0.1323, 0.1298, 0.1295, 0.1295], 'precisions': [0.0, 0.0, 0.0007722007722007722, 0.0007722007722007722, 0.0015444015444015444, 0.0015444015444015444, 0.0015444015444015444, 0.0023166023166023165, 0.0023166023166023165, 0.004633204633204633, 0.005405405405405406, 0.008494208494208495, 0.012355212355212355, 0.013127413127413128, 0.013899613899613899, 0.01698841698841699, 0.022393822393822392, 0.023166023166023165, 0.023166023166023165, 0.023938223938223938, 0.025482625482625483, 0.02702702702702703, 0.03088803088803089, 0.03166023166023166, 0.03397683397683398, 0.03552123552123552, 0.036293436293436294, 0.036293436293436294, 0.036293436293436294, 0.036293436293436294, 0.036293436293436294, 0.037065637065637064, 0.037065637065637064, 0.03861003861003861, 0.039382239382239385, 0.039382239382239385, 0.039382239382239385, 0.039382239382239385, 0.039382239382239385, 0.039382239382239385, 0.039382239382239385, 0.040154440154440155, 0.040154440154440155, 0.040926640926640924, 0.040926640926640924, 0.04247104247104247, 0.043243243243243246, 0.044015444015444015, 0.044787644787644784, 0.044787644787644784, 0.044787644787644784, 0.044787644787644784, 0.044787644787644784, 0.04633204633204633, 0.04633204633204633, 0.047104247104247106, 0.047104247104247106, 0.04864864864864865, 0.04942084942084942, 0.04942084942084942, 0.05019305019305019, 0.050965250965250966, 0.050965250965250966, 0.05328185328185328, 0.05405405405405406, 0.05405405405405406, 0.054826254826254826, 0.055598455598455596, 0.055598455598455596, 0.05637065637065637, 0.05637065637065637, 0.05868725868725869, 0.06023166023166023, 0.06486486486486487, 0.06640926640926641, 0.07104247104247104, 0.07258687258687259, 0.07644787644787644, 0.08108108108108109, 0.08648648648648649, 0.0918918918918919, 0.10501930501930502, 0.12586872586872586, 0.15366795366795366, 0.18146718146718147, 0.24092664092664093, 0.3328185328185328, 0.4548262548262548, 0.5644787644787644, 0.6517374517374518, 0.7575289575289575, 0.8416988416988417, 0.9104247104247104, 0.9436293436293436, 0.9706563706563707, 0.9822393822393822, 0.9861003861003861, 0.9969111969111969, 1.0, 1.0, 1.0], 'recalls': [0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9722222222222222, 0.975609756097561, 0.9761904761904762, 0.9777777777777777, 0.9787234042553191, 0.9791666666666666, 0.9791666666666666, 0.9791666666666666, 0.9791666666666666, 0.9791666666666666, 0.9795918367346939, 0.96, 0.9615384615384616, 0.9444444444444444, 0.9272727272727272, 0.9107142857142857, 0.9107142857142857, 0.9107142857142857, 0.8947368421052632, 0.8793103448275862, 0.8524590163934426, 0.8387096774193549, 0.8412698412698413, 0.828125, 0.8208955223880597, 0.7777777777777778, 0.75, 0.7160493827160493, 0.7073170731707317, 0.6590909090909091, 0.651685393258427, 0.6170212765957447, 0.6122448979591837, 0.6060606060606061, 0.6039603960396039, 0.5980392156862745, 0.5943396226415094, 0.5981308411214953, 0.5981308411214953, 0.5508474576271186, 0.5038167938931297, 0.4782608695652174, 0.46621621621621623, 0.44871794871794873, 0.4430379746835443, 0.4382716049382716, 0.43373493975903615, 0.42105263157894735, 0.4101123595505618, 0.3924731182795699, 0.38578680203045684, 0.36792452830188677, 0.34710743801652894, 0.3173431734317343, 0.304635761589404, 0.29102167182662536, 0.27197802197802196, 0.26184538653366585, 0.24615384615384617, 0.2328767123287671, 0.21794871794871795, 0.20737913486005088, 0.1991991991991992, 0.1857707509881423, 0.17180616740088106, 0.161242050130939, 0.1565240499601382, 0.14884952148238648, 0.1440273037542662, 0.13905031892274983, 0.1356227448052756, 0.13329564725833803, 0.13101747614452663, 0.1299090533278214, 0.12916328188464662, 0.12879475542107918, 0.1295664391810518, 0.12953886165849754, 0.1295, 0.1295], 'F1s': [0, 0, 0.0015432098765432098, 0.0015432098765432098, 0.003084040092521203, 0.003084040092521203, 0.003084040092521203, 0.004622496147919876, 0.004622496147919876, 0.009223674096848577, 0.010752688172043012, 0.016845329249617153, 0.024408848207475208, 0.025914634146341466, 0.027418126428027417, 0.03340926347760061, 0.043806646525679754, 0.045283018867924525, 0.045283018867924525, 0.04675716440422322, 0.04969879518072289, 0.05259203606311044, 0.05988023952095809, 0.061331338818249814, 0.06567164179104479, 0.06855439642324888, 0.06999255398361877, 0.06999255398361877, 0.06999255398361877, 0.06999255398361877, 0.06999255398361877, 0.07142857142857142, 0.07137546468401487, 0.07423904974016332, 0.07561156412157154, 0.07555555555555557, 0.07549962990377498, 0.07549962990377498, 0.07549962990377498, 0.07544378698224852, 0.07538802660753882, 0.07669616519174041, 0.07663964627855564, 0.0780559646539028, 0.07799852832965415, 0.08076358296622613, 0.08193123628383321, 0.08315098468271334, 0.08430232558139535, 0.08424110384894698, 0.08387563268257411, 0.0838150289017341, 0.08351331893448524, 0.08614501076812633, 0.0860832137733142, 0.08739255014326648, 0.08732999284180386, 0.08993576017130622, 0.09129814550641939, 0.09129814550641939, 0.09200283085633404, 0.09256661991584852, 0.09211444521981857, 0.09563409563409564, 0.09648518263266713, 0.09635237439779766, 0.09746053534660261, 0.09856262833675564, 0.09822646657571622, 0.0991174473862865, 0.09858203916272788, 0.10187667560321714, 0.10351692103516921, 0.10930383864671439, 0.10983397190293742, 0.11521603005635567, 0.1161928306551298, 0.11934900542495479, 0.12382075471698115, 0.128, 0.13178294573643412, 0.14174048983845755, 0.15665545410860163, 0.17349607672188316, 0.18359375, 0.20057859209257473, 0.21723790322580644, 0.23289837880585212, 0.23557847244601995, 0.23591893780573023, 0.23497005988023953, 0.23360480068581227, 0.2325443786982248, 0.23008849557522124, 0.22914957615531856, 0.2283047653235215, 0.22783229259589655, 0.22932764899191757, 0.22936592277718737, 0.22930500221336877, 0.22930500221336877], 'logLoss': 0.3804730202285212}]}}